$\newcommand{\To}{\Rightarrow}$

In [1]:
# Initialize to directory holpy. Run this only once!
import os
os.chdir('..')

In [2]:
from kernel.type import Type, STVar, TVar, TConst, TFun, BoolType, NatType, TyInst
from data.list import ListType
from syntax import parser
from syntax.settings import settings
from logic import basic

basic.load_theory('list')

## Types

In higher-order logic, every term has a type. Common types include booleans, natural numbers, functions, lists, and so on. We also need the concept of *type variables*. Types are implemented in `kernel/type.py`.

Booleans and natural numbers are type constants. They can be constructed as follows:

In [3]:
print(TConst("bool"))

bool


In [4]:
print(TConst("nat"))

nat


We use `BoolType` as a shorthand for `TConst("bool")`, and `NatType` as a shorthand for `TConst("nat")`.

In [5]:
print(BoolType)
print(NatType)

bool
nat


Functions is a very important class of types. Given any two types $A$ and $B$, the type $A \To B$ represents functions from $A$ to $B$. For example, the type $nat \To bool$ represents functions from natural numbers to booleans, or in other words, properties of natural numbers. This type is constructed as follows:

In [6]:
print(TConst("fun", NatType, BoolType))

nat => bool


A shortcut to construct function types is to use `TFun`:

In [7]:
print(TFun(NatType, BoolType))

nat => bool


A key concept for dealing with function types is *currying*. It allows us to represent functions of multiple arguments. For example, the type of functions taking two natural numbers as arguments, and output one natural number, is given by $nat \To (nat \To nat)$. Note this is very different from $(nat \To nat) \To nat$. Since the former is used more frequently, we have the convention that the operator $\To$ associates to the right, so the former type is simply written as $nat \To nat \To nat$. In general, the type $A_1 \To \cdots \To A_n \To C$ can be read as: functions taking arguments of type $A_1,\dots A_n$ as input, and output a value of type $C$.

In [8]:
print(TFun(NatType, TFun(NatType, NatType)))

nat => nat => nat


This occurs so frequently that `TFun` supports any number of arguments as follows:

In [9]:
print(TFun(NatType, NatType, NatType))

nat => nat => nat


Functions are not the only types with arguments. Given any type $A$, we can form the type of (finite) lists with entries in $A$:

In [10]:
print(TConst("list", NatType))

nat list


Construction of list types can be abbreviated as follows:

In [11]:
print(ListType(NatType))

nat list


All these can be combined in arbitrary ways. For example, the following is a type representing lists of functions that take a list of natural numbers as input, and returns a natural number:

In [12]:
print(ListType(TFun(ListType(NatType), NatType)))

(nat list => nat) list


A few methods are defined for working with function types:
- `is_fun()` returns whether the type is a function type.
- Given a type $A \To B$, `domain_type()` returns $A$ and `range_type()` returns $B$.
- Given a type $A_1 \To\cdots\To A_n\To B$, `strip_type()` returns the pair $[A_1,\dots,A_n], B$.

In [13]:
print(BoolType.is_fun())

False


In [14]:
a = TFun(NatType, BoolType)
print(a.is_fun())
print(a.domain_type())
print(a.range_type())
print(a.strip_type())

True
nat
bool
([TConst(nat, [])], TConst(bool, []))


In [15]:
b = TFun(NatType, NatType, BoolType)
print(b.is_fun())
print(b.domain_type())
print(b.range_type())
print(b.strip_type())

True
nat
nat => bool
([TConst(nat, []), TConst(nat, [])], TConst(bool, []))


## Type variables

A *type variable* is a variable that can stand in for any type. It can be thought of as an arbitrary but fixed type in the current context. We follow the convention of writing a type variable with name $a$ as 'a. Type variables are constructed as follows:

In [16]:
print(TVar('a'))

'a


A *schematic type variable* is a variable that can be replaced by any type. It usually occurs in the context of statements of theorems, stating that the theorem should be true for any type. We follow the convention of writing a schematic type variable with name $a$ as `?'a`. Type variables are constructed as follows:

In [17]:
print(STVar("a"))

?'a


Type variables can be used as arguments to type constructors. For example, the following type represents all functions from schematic type variables $a$ to $b$:

In [18]:
print(TFun(STVar("a"), STVar("b")))

?'a => ?'b


Next, we introduce the important concepts of *substitution* and *matching*. A type with schematic type variables can be considered as a *pattern* for producing types. If each schematic type variable in the pattern is assigned a concrete value, the pattern can be *instantiated* to a concrete type. We illustrate this with some examples.

In [19]:
p = TFun(STVar("a"), STVar("b"))
print(p)
print(p.subst(a=NatType, b=BoolType))
print(p.subst(a=TFun(NatType, NatType), b=BoolType))
print(p.subst(a=NatType, b=TFun(NatType, BoolType)))

?'a => ?'b
nat => bool
(nat => nat) => bool
nat => nat => bool


In fact, we can assign a type variable to another type containing schematic type variables. Note all substitutions are performed at the same time.

In [20]:
print(p.subst(a=STVar("b"), b=STVar("a")))

?'b => ?'a


Matching can be considered as the dual of substitution. Given a pattern $p$ (a type containing type variables) and a type $t$ (with or without type variables), it determines whether $p$ can be instantiated to $t$, and returns the assignment of type variables in $p$ if it is possible.

In [21]:
print(p.match(TFun(NatType, BoolType)))

'a := nat, 'b := bool


If it is impossible to instantiate $p$ to $t$, the match function throws a `TypeMatchException`:

In [22]:
p.match(NatType)  # raises TypeMatchException

TypeMatchException: Unable to match ?'a => ?'b with nat

Note the same type variable can appear multiple times in a pattern. During matching, each occurrence of the type variable must be assigned to the same type.

In [23]:
q = TFun(ListType(STVar("a")), STVar("a"))
print(q)
print(q.subst(a=NatType))
print(q.match(TFun(ListType(NatType), NatType)))

?'a list => ?'a
nat list => nat
'a := nat


Here is an example of a matching that failed because the two occurrences of `'a` correspond to different types ($nat$ and $bool$).

In [24]:
q.match(TFun(ListType(NatType), BoolType))  # raises TypeMatchException

TypeMatchException: Unable to match bool with nat

## Class hierarchy

The class `Type` is the parent class of all type objects, with `STVar`, `TVar` and `TConst` inheriting from it.

In [25]:
assert isinstance(TConst('bool'), Type)
assert isinstance(TVar('a'), Type)
assert isinstance(STVar('a'), Type)

The functions `is_stvar()`, `is_tvar()` and `is_tconst()` should be used to distinguish between different types.

In [26]:
assert TConst('bool').is_tconst()
assert not TConst('bool').is_tvar()
assert TVar('a').is_tvar()
assert not TVar('a').is_tconst()

The constructor for `Type` can read a string as input, and parses the string as a type:

In [27]:
print(Type('nat'))
print(Type('nat => nat list'))
print(Type('(nat => nat) list'))

nat
nat => nat list
(nat => nat) list


The field `name` can be used to access the name of a type variable or constructor. `args` can be used to access the list of arguments (returned as a tuple):

In [28]:
a = TVar("a")
print(a.name)

b = ListType(NatType)
print(b.name)
print(b.args)

a
list
(TConst(nat, []),)


The following functions return various classes of types appearing in a type:
* `get_tsubs()` returns the list of all types appearing in a type (including itself).
* `get_tvars()` returns the list of type variables in a type.
* `get_stvars()` returns the list of schematic type variables in a type. 

In [29]:
a = TFun(TVar("a"), STVar("b"))
print(a)
print(a.get_tsubs())
print(a.get_tvars())
print(a.get_stvars())

'a => ?'b
[TConst(fun, [TVar(a), STVar(b)]), TVar(a), STVar(b)]
[TVar(a)]
[STVar(b)]
